In [1]:
# 基本ライブラリ
import pandas as pd
import pandas.io.sql as psql
import numpy as np
import numpy.random as rd
import gc
import multiprocessing as mp
import os
import sys
import pickle
from collections import defaultdict
from glob import glob
import math
from datetime import datetime as dt
from pathlib import Path
import scipy.stats as st
import re
import shutil
from tqdm import tqdm_notebook as tqdm
import datetime
ts_conv = np.vectorize(datetime.datetime.fromtimestamp) # 秒ut(10桁) ⇒ 日付

# グラフ描画系
import matplotlib
from matplotlib import font_manager
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from matplotlib import rc

from matplotlib import animation as ani
from IPython.display import Image

plt.rcParams["patch.force_edgecolor"] = True
#rc('text', usetex=True)
from IPython.display import display # Allows the use of display() for DataFrames
import seaborn as sns
sns.set(style="whitegrid", palette="muted", color_codes=True)
sns.set_style("whitegrid", {'grid.linestyle': '--'})
red = sns.xkcd_rgb["light red"]
green = sns.xkcd_rgb["medium green"]
blue = sns.xkcd_rgb["denim blue"]

#カラム内の文字数。デフォルトは50
pd.set_option("display.max_colwidth", 100)

#行数
pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)
#
pd.options.display.float_format = '{:,.5f}'.format

%matplotlib inline

In [2]:
import lightgbm as lgb
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold, KFold, RepeatedKFold

from sklearn import metrics
import json

import warnings
warnings.filterwarnings("ignore")


sys.path.append('..')
from lib.line_notif import send_message
from lib.utils import reduce_mem_usage, current_time, unpickle, to_pickle
from lib.utils import one_hot_encoder, apply_agg, multi_combine_categorical_feature
from lib.utils import import_data, get_split_indexer 

In [3]:
sub = pd.read_csv('../input/sample_submission.csv')

In [4]:
DATA_VERSION = "v001"
TRIAL_NO = "005"
save_path = Path(f"../processed/{DATA_VERSION}")
model_path = Path(f"../model/{DATA_VERSION}_{TRIAL_NO}")
submit_path = Path(f"../submit/{DATA_VERSION}_{TRIAL_NO}")

In [5]:


print("start loading...")
test  = unpickle(save_path/"test_002.df.pkl", )
print(f"test loaded.")
test.set_index("id", inplace=True)
print(f"test: {test.shape}")
print(test.shape,)
groups = unpickle(save_path/"lbl_molecule_name.pkl", )

def get_test_data_for_predict(mol_type):
    # mol_type = 1 # int(sys.argv[1]) #1
    print(f"mol_type: {mol_type}")

    if mol_type==0:
        pass
    elif mol_type==1:
        train_type_cut = np.load("../processed/v001/train_type1_cut.npy", )
        test_type_cut = np.load("../processed/v001/test_type1_cut.npy", )
    elif mol_type==2:
        train_type_cut = np.load("../processed/v001/train_type2_cut.npy", )
        test_type_cut = np.load("../processed/v001/test_type2_cut.npy", )
    elif mol_type==3:
        test_type_cut = np.load("../processed/v001/test_type3_cut.npy", )
        train_type_cut = np.load("../processed/v001/train_type3_cut.npy", )
    else:
        assert False, f"mol_type should be 0, 1, 2, 3. mol_type: {mol_type}"

#     if mol_type in [1,2,3]:
#         test_  = test[test_type_cut]
    return test[test_type_cut]

start loading...
test loaded.
test: (2505542, 448)
(2505542, 448)


In [ ]:
sub_list = []
n_folds = 5
iterations = [29000, 7000, 20000]

for mol_type, iteration in zip([1,2,3], iterations):
    print(f"mol_type: {mol_type}")
    test_ = get_test_data_for_predict(mol_type=mol_type)
    #bst_list = []
    preds_test = np.zeros((test_.shape[0], n_folds))
    for fold_ in tqdm(range(n_folds)):
        saved_model_path = f'../model/{DATA_VERSION}_{TRIAL_NO}/booster_{mol_type}_{fold_:02d}_{iteration}.model'
        bst = lgb.Booster(model_file=saved_model_path)
        #bst_list += [bst]
        preds_test[:, fold_] = bst.predict(test_)

    sub = pd.DataFrame(index=test_.index)
    sub["scalar_coupling_constant"] = preds_test.mean(axis=1)    
    sub_list.append(sub)
    
sub_all = pd.concat(sub_list, axis=0)    
sub_all.to_csv(submit_path/"sub_all.csv")

mol_type: 1
mol_type: 1
